<a href="https://colab.research.google.com/github/Charlotte-99/Y3Project/blob/main/deepsphere_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Healpy and DeepSphere

## Importing the modules
First we mount our google drive (this will bring up an authentication process).

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

We can then change directory to our main Google drive folder.

In [ ]:
os.chdir('/content/drive/My Drive')

Then we clone the GitHub repository and change directory to this folder.
This should only need to be done this one time, afterwards we can comment out the cloning line.

In [ ]:
#!git clone https://github.com/deepsphere/deepsphere-cosmo-tf2.git
os.chdir('/content/drive/My Drive/deepsphere-cosmo-tf2')

We then have to install the package requirements using the .txt file.

In [ ]:
!pip install -r requirements.txt

Then we install the deepsphere package.

After running this cell, restart the runtime.

In [ ]:
!pip install -e .

Then we can import modules (and make sure the directory is correct).

In [ ]:
import deepsphere
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np 
import os
import healpy as hp


from google.colab import files
from deepsphere import HealpyGCNN
import deepsphere.healpy_layers as hp_layer
from matplotlib.colors import ListedColormap

os.chdir('/content/drive/My Drive/deepsphere-cosmo-tf2')

## DeepSphere CNN
Can then play around with tutorials.

In [ ]:
# Upload files (this will upload to your drive folder)
files.upload()

### Generating Maps
We will use the healpy library to generate sky maps for different power spectra. 

First we will load the power spectra and colour map.

In [ ]:
# Load spectra as np arrays
power_0_02 = np.loadtxt('camb_0_02.txt')
power_0_03 = np.loadtxt('camb_0_03.txt')
power_0_04 = np.loadtxt('camb_0_04.txt')
power_0_05 = np.loadtxt('camb_0_05.txt')

# Load colourmap
cmap = ListedColormap(np.loadtxt('planck_map.txt')/255.)

We can plot these power spectra against each other to see what they look like.

In [ ]:
fig, ax = plt.subplots()

ls = power_0_02[:, 0]
ax.plot(ls, power_0_02[:, 1], label='$\Omega_b h^2 = 0.02$')
ax.plot(ls, power_0_03[:, 1], label='$\Omega_b h^2 = 0.03$')
ax.plot(ls, power_0_04[:, 1], label='$\Omega_b h^2 = 0.04$')
ax.plot(ls, power_0_05[:, 1], label='$\Omega_b h^2 = 0.05$')
ax.set_xlabel('$l$')
ax.set_ylabel('$l(l+1)C_l$')
plt.legend()

We can then use healpy's synfast function to generate sky maps for each of these spectra. First we will plot some higher resolution examples.

In [ ]:
lmax = 2200
nside = 512

# Function to generate a map
def generate_map(lmax, nside, power_spectrum):
  cls_input = np.divide(power_spectrum[:, 1],
                        power_spectrum[:, 0]*(power_spectrum[:, 0] +1))
  cls = np.concatenate([np.zeros(2), cls_input])
  map = hp.sphtfunc.synfast(cls, nside=nside, lmax=lmax, new=True)
  return map

# Plot a map for each of the spectra
for spectrum in [power_0_02,  power_0_03, power_0_04, power_0_05]:
  hp.mollview(generate_map(lmax, nside, spectrum), cmap=cmap)

We can then generate some (lower) resolution maps for each of these spectra and save them to a file. We will generate 100 maps for each of the spectra.

In [ ]:
lmax = 2200
nside = 64

num_maps = 100
num_spectra = 4
num_pixels = hp.nside2npix(nside)

# Initialise empty array that we will fill
maps = np.empty((num_maps*num_spectra, num_pixels))

# Generate maps
i = 0  # index
for spectrum in [power_0_02,  power_0_03, power_0_04, power_0_05]:
  for n in range(num_maps):
    maps[i] = generate_map(lmax, nside, spectrum)
    i += 1
  print('done spectrum')


We will now have an array of shape (num_maps x num_spectra, num_pixels). We can reshape this into (num_maps x num_spectra, num_pixels, 1).


In [ ]:
maps = np.reshape(maps, (num_maps*num_spectra, num_pixels, 1))

We can then save this to a file together with the y labels.

In [ ]:
classes = np.concatenate([np.zeros(num_maps), np.ones(num_maps),
                          np.ones(num_maps)*2, np.ones(num_maps)*3])

np.savez_compressed('full_maps.npz', maps=maps, classes=classes)

### Using DeepSphere

In this section we will experiment with the DeepSphere CNN.

In [ ]:
# Load data
data = np.load('full_maps.npz')

We will first experiment with only two of the classes (0.2 and 0.5) to give the largest differences.

Here we split our dataset into test and train sets.

In [ ]:
from sklearn.model_selection import train_test_split
x_raw = np.concatenate([data['maps'][:100], data['maps'][300:]])[..., np.newaxis]
y_raw = np.concatenate([np.zeros(100), np.ones(100)])

np.random.RandomState(11).shuffle(x_raw)
np.random.RandomState(11).shuffle(y_raw)

x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.2,
                                                    random_state=11)

Here we will experiment with a basic DeepSphere CNN without using survey masks. The guide used for this section is the quick start tutorial on the DeepSphere repo.


In [ ]:
layers = [hp_layer.HealpyChebyshev(K=10, Fout=5, use_bias=True, use_bn=True, 
                                   activation="relu"),
          hp_layer.HealpyPool(p=1),
          hp_layer.HealpyChebyshev(K=10, Fout=5, use_bias=True, use_bn=True, 
                                   activation="relu"),
          hp_layer.HealpyPool(p=1),
          hp_layer.HealpyChebyshev(K=10, Fout=5, use_bias=True, use_bn=True, 
                                   activation="relu"),
          hp_layer.HealpyPool(p=1),
          hp_layer.HealpyChebyshev(K=10, Fout=2),
          tf.keras.layers.Lambda(lambda x: tf.nn.softmax(tf.reduce_mean(x, axis=1),
                                                         axis=-1))]



nside = 64
indices = np.arange(hp.nside2npix(nside))

tf.keras.backend.clear_session()
model = HealpyGCNN(nside=nside, indices=indices, layers=layers, n_neighbors=20)
batch_size = 16
model.build(input_shape=(None, len(indices), 1))
model.summary(110)

model.compile(optimizer=tf.keras.optimizers.Adam(0.1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)


We can then train the model.

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=50,
    validation_data=(x_test, y_test),
)

Then plot the loss and accuracy.

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history["loss"], label="training")
plt.plot(history.history["val_loss"], label="validation")
plt.grid()
plt.yscale("log")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['sparse_categorical_accuracy'], label="training")
plt.plot(history.history['val_sparse_categorical_accuracy'], label="validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")